<h1><center><font color='red'>Argumentation Project</font></center></h1>

<h2><font color='green'>Guidelines</font></h2>

Here you will find the three contexts: complete, (with and without id) and the subcontexts for and against (Thanks Laurine!).<br>

The original micro texts and arg structures are found here: https://github.com/peldszus/arg-microtexts-multilayer<br>

DESCRIPTION: cf. slides<br>

Questions & texts: see .xml in https://github.com/peldszus/arg-microtexts/tree/master/corpus/en<br>

<br>Arg. structures: see .pdf in the same page

<br>There is an additional file (arg_attr_patterns.json) that contains the correspondence between the structure identifier arg and the corresponding structure:

<br>=> json file {k:v} with k the identifier, and v the structure (a character string) in the following format:

<ul>
    <li>t # arg_id // the first line contains the structure identifier (arg_id)</li>
    <li>v v_id v_label // each line starting with v describes a node v_id and its label v_label (no label on the nodes, I put "_")</li>
    <li>e src_id trg_id e_label // each line starting with e describes an arc between src_id and trg_id (two previously defined nodes) and its label e_label</li>
</ul>

<br>The goal of the project is to classify microtexts w.r.t. the argumentation structures that they contain. The classification should rely on the hypotheses (for, against, falsified generalizations) that you'll mine, and an analysis is expected. Reference to the descriptions given in the page above is required. You should sample a few examples for testing your classifier. Compare with more "classical" pattern mining approaches (seen last year). This is an exploratory project!

<br>You'll have to form groups of a maximum of 3 (you can choose the same groups), and write a  short report with your findings and analysis.

<br>Following the discussion with the student representatives, the new Deadline is 28 of February 2022 (midnight)!

<h2><font color='green'>Initialisation</font></h2>

In [20]:
# Importation cell
import pandas as pd
import typing as t
import json
import os
from lxml import etree


<h2><font color='green'>Dataframe creation and observations</font></h2>

<h3><font color='orange'><center>arg_context_con.csv<center></font></h3>

In [6]:
df_arg_context_con = pd.read_csv('Data/arg_context_con.csv',sep=';')
df_arg_context_con.head()

,Unnamed: 0,a0,a1,a2,a3,a4,a5,a6,a7,a8,...,a88,a89,a90,a91,a92,a93,a94,a95,a96,a97
0,micro_k017_con,1,1,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,micro_b027_con,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,micro_k003_con,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,micro_k016_con,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,micro_b018_con,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3><font color='orange'><center>arg_context_glob.csv<center></font></h3>

In [7]:
df_arg_context_glob = pd.read_csv('Data/arg_context_glob.csv',sep=';')
df_arg_context_glob.head()

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a89,a90,a91,a92,a93,a94,a95,a96,a97,Unnamed: 98
0,1,1,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,NaN
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,NaN
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,1,1,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


<h3><font color='orange'><center>arg_context_pro.csv<center></font></h3>

In [8]:
df_arg_context_glob = pd.read_csv('Data/arg_context_pro.csv',sep=';')
df_arg_context_glob.head()

,Unnamed: 0,a0,a1,a2,a3,a4,a5,a6,a7,a8,...,a88,a89,a90,a91,a92,a93,a94,a95,a96,a97
0,micro_b033_pro,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,micro_b026_pro,1,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,micro_k002_pro,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,micro_b032_pro,1,0,0,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
4,micro_b024_pro,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3><font color='orange'><center>complete_context_withlabels.csv<center></font></h3>

In [9]:
df_arg_context_glob = pd.read_csv('Data/complete_context_withlabels.csv',sep=';')
df_arg_context_glob.head()

,Unnamed: 0,a0,a1,a2,a3,a4,a5,a6,a7,a8,...,a88,a89,a90,a91,a92,a93,a94,a95,a96,a97
0,micro_k017_con,1,1,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,micro_b033_pro,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,micro_b027_con,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,micro_k003_con,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,micro_b026_pro,1,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


<h2><font color='green'>Extraction pdf</font></h2>

In [37]:
# we try to get all xml file in only one liste
l_xml = [f'"en/{elt}"' for elt in os.listdir('en') if elt[-3:] == 'xml'] # 112 files
print(l_xml[0])
tree = etree.parse('en/micro_b001.xml')
for elt in tree.getroot().getchildren():
    print(elt.attrib)

"en/micro_b001.xml"
{'id': 'e1'}
{'id': 'e2'}
{'id': 'e3'}
{'id': 'e4'}
{'id': 'e5'}
{'id': 'a1', 'type': 'opp'}
{'id': 'a2', 'type': 'opp'}
{'id': 'a3', 'type': 'pro'}
{'id': 'a4', 'type': 'pro'}
{'id': 'a5', 'type': 'pro'}
{'id': 'c6', 'src': 'e1', 'trg': 'a1', 'type': 'seg'}
{'id': 'c7', 'src': 'e2', 'trg': 'a2', 'type': 'seg'}
{'id': 'c8', 'src': 'e3', 'trg': 'a3', 'type': 'seg'}
{'id': 'c9', 'src': 'e4', 'trg': 'a4', 'type': 'seg'}
{'id': 'c10', 'src': 'e5', 'trg': 'a5', 'type': 'seg'}
{'id': 'c1', 'src': 'a1', 'trg': 'a5', 'type': 'reb'}
{'id': 'c2', 'src': 'a2', 'trg': 'a1', 'type': 'sup'}
{'id': 'c3', 'src': 'a3', 'trg': 'c1', 'type': 'und'}
{'id': 'c4', 'src': 'a4', 'trg': 'c3', 'type': 'add'}
